# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [36]:
# Dependencies and Setup
import geoviews as gv
import cartopy.crs as ccrs
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [37]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,5.73,87,100,3.09,US,1684152873
1,1,konkwesso,10.8469,4.0984,37.96,32,100,3.62,NG,1684153102
2,2,vadso,70.0744,29.7487,20.49,47,0,4.63,NO,1684153104
3,3,waitangi,-43.9535,-176.5597,13.92,74,37,8.36,NZ,1684152692
4,4,lautoka,-17.6167,177.4667,20.92,88,75,0.00,FJ,1684153106


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:
%%capture --no-display

# Configure the map plot
map_plot_1=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

ideal_weather=city_data_df.loc[(city_data_df["Max Temp"] <= 27) & (city_data_df["Max Temp"] >= 21)&
                               (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values
ideal_weather_df=ideal_weather.dropna(axis=0 , how="any")

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,arraial do cabo,-22.9661,-42.0278,22.98,83,0,2.57,BR,1684152973
136,136,vila velha,-20.3297,-40.2925,24.97,80,0,1.54,BR,1684152876
279,279,sao vicente,-23.9631,-46.3919,21.82,73,0,1.03,BR,1684153441
444,444,pascagoula,30.3658,-88.5561,24.40,91,0,0.00,US,1684153574
498,498,mancio lima,-7.6142,-72.8958,24.04,94,0,0.00,BR,1684153736


### Step 3: Create a new DataFrame called `hotel_df`.

In [40]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_weather_df[["City", "Country","Lat","Lng" , "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

/var/folders/gz/yqwhjwdd6c75tl8kh7k5sr700000gn/T/ipykernel_22682/3367169976.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
18,arraial do cabo,BR,-22.9661,-42.0278,83,
136,vila velha,BR,-20.3297,-40.2925,80,
279,sao vicente,BR,-23.9631,-46.3919,73,
444,pascagoula,US,30.3658,-88.5561,91,
498,mancio lima,BR,-7.6142,-72.8958,94,
527,la mira,MX,18.0350,-102.3266,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": 'accommodation.hotel',
          "limit": 1,
          "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

vila velha - nearest hotel: Hotel Prainha
sao vicente - nearest hotel: Pousada Vitória
pascagoula - nearest hotel: Grand Magnolia
mancio lima - nearest hotel: No hotel found
la mira - nearest hotel: Hotel La Loma


,City,Country,Lat,Lng,Humidity,Hotel Name
18,arraial do cabo,BR,-22.9661,-42.0278,83,No hotel found
136,vila velha,BR,-20.3297,-40.2925,80,Hotel Prainha
279,sao vicente,BR,-23.9631,-46.3919,73,Pousada Vitória
444,pascagoula,US,30.3658,-88.5561,91,Grand Magnolia
498,mancio lima,BR,-7.6142,-72.8958,94,No hotel found
527,la mira,MX,18.0350,-102.3266,68,Hotel La Loma


Starting hotel search
arraial do cabo - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
sao vicente - nearest hotel: Pousada Vitória
pascagoula - nearest hotel: Grand Magnolia
mancio lima - nearest hotel: No hotel found
la mira - nearest hotel: Hotel La Loma


,City,Country,Lat,Lng,Humidity,Hotel Name
18,arraial do cabo,BR,-22.9661,-42.0278,83,No hotel found
136,vila velha,BR,-20.3297,-40.2925,80,Hotel Prainha
279,sao vicente,BR,-23.9631,-46.3919,73,Pousada Vitória
444,pascagoula,US,30.3658,-88.5561,91,Grand Magnolia
498,mancio lima,BR,-7.6142,-72.8958,94,No hotel found
527,la mira,MX,18.0350,-102.3266,68,Hotel La Loma


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha=0.5,
    hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)